# Imbalanced Data

- skip_exec: true


In [ ]:
from imblearn.datasets import make_imbalance
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split


In [ ]:
X, y = fetch_openml(data_id=1119, as_frame=True, return_X_y=True)
X = X.select_dtypes(include="number")
X, y = make_imbalance(X, y, sampling_strategy={">50K": 300}, random_state=1)


/Users/henrydashwood/.pyenv/versions/3.11.1/envs/py3111/lib/python3.11/site-packages/sklearn/datasets/_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [ ]:
X


,age,fnlwgt:,education-num:,capital-gain:,capital-loss:,hours-per-week:
0,39.0,77516.0,13.0,2174.0,0.0,40.0
1,50.0,83311.0,13.0,0.0,0.0,13.0
2,38.0,215646.0,9.0,0.0,0.0,40.0
3,53.0,234721.0,7.0,0.0,0.0,40.0
4,28.0,338409.0,13.0,0.0,0.0,40.0
...,...,...,...,...,...,...
25015,52.0,254211.0,14.0,15024.0,0.0,60.0
25016,44.0,377018.0,11.0,0.0,0.0,40.0
25017,36.0,114605.0,11.0,0.0,0.0,40.0
25018,37.0,212005.0,11.0,0.0,0.0,40.0


In [ ]:
y


0        <=50K
1        <=50K
2        <=50K
3        <=50K
4        <=50K
         ...  
25015     >50K
25016     >50K
25017     >50K
25018     >50K
25019     >50K
Name: class, Length: 25020, dtype: category
Categories (2, object): ['<=50K', '>50K']

In [ ]:
y.value_counts(normalize=True)


<=50K    0.98801
>50K     0.01199
Name: class, dtype: float64

In [ ]:
X, X_left_out, y, y_left_out = train_test_split(X, y, stratify=y, random_state=0)